In [1]:
#! pip uninstall virny

In [2]:
! pip install fairlearn

zsh:1: /usr/local/bin/pip: bad interpreter: /System/Library/Frameworks/Python.framework/Versions/2.7/Resources/Python.app/Contents/MacOS/Python: no such file or directory

[notice] A new release of pip available: 22.2.1 -> 23.1.1
[notice] To update, run: python3.9 -m pip install --upgrade pip


In [3]:
from sklearn.model_selection import train_test_split

import pandas as pd
import numpy as np
import scipy as sp 
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from data_loader import CompasDataset, ACSEmploymentDataset
from utils import predict_ensemble, predict_CIID
from utils import partition_by_group_intersectional, partition_by_group_binary, set_protected_groups

/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/requests/__init__.py:102: RequestsDependencyWarning: urllib3 (1.26.6) or chardet (5.1.0)/charset_normalizer (2.0.4) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({})/charset_normalizer ({}) doesn't match a supported "


In [4]:
SEED = 42

In [5]:
dataset = CompasDataset()

In [6]:
train, test_, = train_test_split(dataset.dataset, test_size=0.2, random_state=42)
val, test, = train_test_split(test_, test_size=0.5, random_state=42)

train.shape, val.shape, test.shape


((4222, 13), (528, 13), (528, 13))

In [7]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from config import MODELS_CONFIG

In [8]:
clf = RandomForestClassifier(max_depth= 4, max_features = 0.6, min_samples_leaf= 1, n_estimators=500, random_state=SEED)
clf.fit(train[dataset.features], train[dataset.target])
clf.score(val[dataset.features], val[dataset.target])

0.7102272727272727

In [9]:
from fairlearn.postprocessing import ThresholdOptimizer

In [10]:
# 1 is Causian, 0 is others
sensitive_features = train['race'].apply(lambda x: 1 if x == 'Caucasian' else 0)

In [11]:
postprocess_est = ThresholdOptimizer(estimator=clf, constraints="false_negative_rate_parity",
                                     objective="balanced_accuracy_score",prefit=True, predict_method='predict_proba')
postprocess_est.fit(train[dataset.features], train[dataset.target], sensitive_features=sensitive_features)

ThresholdOptimizer(constraints='false_negative_rate_parity',
                   estimator=RandomForestClassifier(max_depth=4,
                                                    max_features=0.6,
                                                    n_estimators=500,
                                                    random_state=42),
                   objective='balanced_accuracy_score',
                   predict_method='predict_proba', prefit=True)

In [12]:
val_sensitive = val['race'].apply(lambda x: 1 if x == 'Caucasian' else 0)
fair_preds = postprocess_est.predict(val[dataset.features], sensitive_features = val_sensitive)

In [13]:
compas_dict = dict({
    'sex': 0, 
    'race': 'Caucasian', 
    'sex&race': None
    })

test_protected_groups = set_protected_groups(test, ['sex', 'race'], [0, 'Caucasian'])

In [14]:
from Virny.virny.analyzers.subgroup_statistical_bias_analyzer import SubgroupStatisticalBiasAnalyzer
from Virny.virny.analyzers.subgroup_variance_analyzer import SubgroupVarianceAnalyzer
from Virny.virny.configs.constants import ModelSetting

In [15]:
ModelSetting.BATCH

<ModelSetting.BATCH: 'batch'>

In [16]:
error_analyser = SubgroupStatisticalBiasAnalyzer(test[dataset.features], test[dataset.target], compas_dict, test_protected_groups)
variance_analyser = SubgroupVarianceAnalyzer(n_estimators=200, base_model=postprocess_est, base_model_name='postprocess', 
                                             bootstrap_fraction=0.8, X_train=train[dataset.features], y_train=train[dataset.target],
                                             X_test=test[dataset.features], y_test=test[dataset.target], target_name=dataset.target,
                                             dataset_name='compas', sensitive_attributes_dct=compas_dict, 
                                             test_protected_groups=test_protected_groups, sensitive_features=sensitive_features)

In [17]:
error_metrics = error_analyser._compute_metrics(test[dataset.target], fair_preds)

In [18]:
variance_metrics = variance_analyser.compute_metrics(save_results=True, result_filename='compas_testing.png')

2023-04-23 21:55:21 abstract_overall_variance_analyzer.py INFO    : Start classifiers testing by bootstrap


Classifiers testing by bootstrap:   0%|          | 0/200 [00:00<?, ?it/s]

ValueError: Found input variables with inconsistent numbers of samples: [3377, 4222]

In [ ]:
error_metrics

{'TPR': 0.5443548387096774,
 'TNR': 0.5035714285714286,
 'PPV': 0.4927007299270073,
 'FNR': 0.45564516129032256,
 'FPR': 0.49642857142857144,
 'Accuracy': 0.5227272727272727,
 'F1': 0.5172413793103449,
 'Selection-Rate': 0.5189393939393939,
 'Positive-Rate': 1.1048387096774193}

In [ ]:
# SubgroupVarianceAnalyzer ---> Generic Pipeline 

#https://github.com/DataResponsibly/Virny/blob/b04cef8c96e0de7903ab515661841b60dcadb413/virny/analyzers/subgroup_variance_analyzer.py#L5